# Q10

In [ ]:
import numpy as np


import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import layers as L

In [ ]:
data = K.datasets.mnist.load_data()

In [ ]:
(X_train, y_train), (X_test, y_test) = data
X_train = X_train.astype(float) / 255.0
X_test = X_test.astype(float) / 255.0

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]


input_shape = X_train.shape[1:]
num_classes = len(set(y_train))

y_train = K.utils.to_categorical(y_train, num_classes)
y_test = K.utils.to_categorical(y_test, num_classes)

In [ ]:
def build_model():
    model = K.Sequential()
    model.add(L.InputLayer(input_shape=input_shape))
    model.add(L.Conv2D(32, 3, 2, activation="relu"))
    model.add(L.BatchNormalization())
    model.add(L.Conv2D(16, 3, 2,activation="relu"))
    model.add(L.BatchNormalization())
    # model.add(L.Conv1D(1, 3, 1, activation="relu"))
    model.add(L.Flatten())
    model.add(L.Dense(10, activation="softmax"))
    return model


In [ ]:
model = build_model()
print(model.summary())
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics="accuracy"
)
history = model.fit(x=X_train, y=y_train, epochs=20, validation_split=0.1, batch_size=128)

In [ ]:
print(model.evaluate(x=X_test, y=y_test))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(8, 14))
plt.sca(axes[0])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


plt.sca(axes[1])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Q9

In [ ]:
import numpy as np


from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder, 
                                   LabelEncoder, 
                                   FunctionTransformer)

import pandas as pd

from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPClassifier

In [ ]:
import os 
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"
!kaggle datasets download -d uciml/forest-cover-type-dataset
!unzip forest-cover-type-dataset.zip

In [ ]:
df = pd.read_csv("covtype.csv")
X = df.loc[:, df.columns != 'Cover_Type'].copy()
Y = df.loc[:, df.columns == 'Cover_Type'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 101)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

num_vars = ["Elevation", 
            "Aspect", 
            "Slope", 
            "Horizontal_Distance_To_Hydrology", 
            "Vertical_Distance_To_Hydrology", 
            "Horizontal_Distance_To_Roadways",
            "Hillshade_9am", 
            "Hillshade_Noon", 
            "Hillshade_3pm", 
            "Horizontal_Distance_To_Fire_Points"]

scaler = StandardScaler()
scaler.fit(X_train.loc[:, num_vars])
X_train.loc[:, num_vars] = scaler.transform(X_train.loc[:, num_vars])
X_test.loc[:, num_vars] = scaler.transform(X_test.loc[:, num_vars])

X_train = X_train.values
X_test = X_test.values

print(X_train.shape, X_test.shape)

In [ ]:
pipe = Pipeline([("clf", MLPClassifier(batch_size=256, verbose=True,))])
pipe.fit(X_train, y_train)
print(accuracy_score(y_test, pipe.predict(X_test)))

In [ ]:
pipe = Pipeline([
                 ("pca", PCA(n_components=0.8)),
                 ("clf", MLPClassifier(batch_size=256, verbose=True,))
                 ])
pipe.fit(X_train, y_train)
print(accuracy_score(y_test, pipe.predict(X_test)))